In [1]:
# Set up

# Import packages
import os, requests
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd

# Download and load data 
from data.get_data import load,download

download()

# Load data
alldat = load()

Loading data. Patience, padawan ...


In [2]:
# Basic information about Steinmetz dataset: descriptive for each session
session = []
mouse_name = []
number_trials = []
total_trials = 0
for i in range(len(alldat)):
    dat = alldat[i]
    session.append(i+1)
    mouse_name.append(dat.get('mouse_name'))
    number_trials.append(len(dat.get('gocue')))
    total_trials += len(dat.get('gocue'))
    
first_dict = {'Session': session, 'Mouse Name': mouse_name, 'Number of Trials': number_trials}
basic_data_info = pd.DataFrame(first_dict, columns=['Session', 'Mouse Name','Number of Trials'])

print(basic_data_info)
print('Total number of trials: ', total_trials)

    Session Mouse Name  Number of Trials
0         1       Cori               214
1         2       Cori               251
2         3       Cori               228
3         4  Forssmann               249
4         5  Forssmann               254
5         6  Forssmann               290
6         7  Forssmann               252
7         8      Hench               250
8         9      Hench               372
9        10      Hench               447
10       11      Hench               342
11       12  Lederberg               340
12       13  Lederberg               300
13       14  Lederberg               268
14       15  Lederberg               404
15       16  Lederberg               280
16       17  Lederberg               224
17       18  Lederberg               316
18       19      Moniz               247
19       20      Moniz               235
20       21      Moniz               124
21       22     Muller               444
22       23     Muller               151
23       24     

In [3]:
#DataFrame for EVERY-SINGLE-TRIAL *clap emoji* (excluding passive trials: just Go (left/right) or NoGo trials)

s = 0
test_sessions = [1, 4, 8, 12, 19, 22, 25, 30, 35]

# Initialize empty list 
session = [] 
session_type = []
mouse_name = []  
trial_number = []
go_trial = []   #either TRUE (Go trial) or FALSE(= NoGo trial)
trial_type = []  # A, B, C, D, or E
stim_loc = []
ideal_resp = []
gocue = [] #dat['gocue']: when the go cue sound was played. 
latency = []
response_time = []
mouse_resp = []
wheel_velocity = []
wheel_acceleration = []
pres_acc = []
feedback_onset = []
feedback_type = []
contrast_left = []
contrast_right = []
contrast_diff = []
pres_difficulty = []

# Loop to fill all lists with data from Steinmetz dataset
for dat in alldat:
    s += 1
    for t in range(len(dat.get('gocue'))):      #just because the length of this field = number of trials for a given session
        # session #
        session.append(s) #session number

        if session[t] == 1 or session[t] == 4 or session[t] == 8 or session[t] == 12 or session[t] == 19 or session[t] == 22 or session[t] == 25 or session[t] == 30 or session[t] == 35:
            session_type.append('test')
        else: 
            session_type.append('train') 
        
        # mouse name
        mouse_name.append(dat.get('mouse_name')) #you know... just the name of the little fellow
        
        # trial number
        trial_number.append(t+1)      #trial number for that particular session

        # whether it was a go trial or not
        go_trial.append((dat.get('contrast_left')[t] != 0) or (dat.get('contrast_right')[t] != 0))
        
        # trial type, stimulus location on pres trial, and ideal response
        if dat.get('contrast_left')[t] == 0 and dat.get('contrast_right')[t] != 0:
            trial_t = 'A'
            stim_l = -1
            ideal_r = -1
        elif dat.get('contrast_left')[t] != 0 and dat.get('contrast_right')[t] == 0:
            trial_t = 'B'
            stim_l = 1
            ideal_r = 1
        elif dat.get('contrast_left')[t] == dat.get('contrast_right')[t] and dat.get('contrast_left')[t] != 0:
            trial_t = 'C'
            stim_l = 2
            ideal_r = np.nan
        elif dat.get('contrast_right')[t] and dat.get('contrast_left')[t] != 0 and dat.get('contrast_right')[t] > dat.get('contrast_left')[t] :
            trial_t = 'D'
            stim_l = 2
            ideal_r = -1
        elif dat.get('contrast_right')[t] and dat.get('contrast_left')[t]  != 0 and dat.get('contrast_right')[t] < dat.get('contrast_left')[t] :
            trial_t = 'E'
            stim_l= 2
            ideal_r = 1
        elif dat.get('contrast_left')[t] == 0 and dat.get('contrast_right')[t] == 0:
            trial_t= 'F'
            stim_l = 0
            ideal_r = 0
        else:
            trial_t = np.nan
            stim_l = np.nan
            ideal_r = np.nan
            
        trial_type.append(trial_t) 
        stim_loc.append(stim_l)
        ideal_resp.append(ideal_r)
        
        # go cue onset
        gocue.append(dat.get('gocue')[t][0]*1000) #the sencond index value is just there so that we can directly access floats from the DF
        
        # mouse response time 
        response_time.append(dat.get('response_time')[t][0]*1000)
        
        # latency 
        latency.append(dat.get('response_time')[t][0]*1000 - dat.get('gocue')[t][0]*1000)

        # append mouse's response
        mouse_resp.append(dat.get('response')[t])
        
        # velocity
        wheel_velocity.append(dat['wheel'][0][0] * .135) 
        
        # wheel acceleration
        wheel_acceleration.append(np.diff(wheel_velocity[t]/10))
    
        # feedback onset 
        feedback_onset.append((dat.get('feedback_time')[t][0]*1000))
        
        # feedback type 
        feedback_type.append(dat.get('feedback_type')[t]) #feedback type: positive (+1) means reward, negative (-1) means white noise burst 
       
        # stimulus contrasts and differences between them
        contrast_left.append(dat.get('contrast_left')[t])
        contrast_right.append(dat.get('contrast_right')[t])
        
        if stim_loc[t] != 0:
            contrast_diff.append((dat.get('contrast_left')[t] - dat.get('contrast_right')[t]).astype(float))
        else:
            contrast_diff.append(np.nan)

        # present difficulty
        if (dat.get('contrast_left')[t] != 0) or (dat.get('contrast_right')[t] != 0):
            pres_difficulty.append(1-np.abs(dat.get('contrast_left')[t]-dat.get('contrast_right')[t]))
        else:
            pres_difficulty.append(np.nan)
            
        # mouse's accuracy on present trial
        if feedback_type[t] == 1: 
            mouse_acc_binary = 1
            pres_acc.append(mouse_acc_binary)
        else:
            mouse_acc_binary = 0
            pres_acc.append(mouse_acc_binary)


In [9]:
type(gocue)

list

In [8]:
len(dat['gocue'])

343